In [1]:
"""
This sample demonstrates how to request and save the market scanner parameters from TWS/IB Gateway
using the IB API. Scanner parameters are returned as an XML string containing a comprehensive list 
of scanning tags, filters, and available codes. This XML data is typically very large (around 2 MB),
so we save it to a file rather than printing it to the terminal.
"""

import os
from ibapi.client import EClient
from ibapi.wrapper import EWrapper


In [2]:

class TestApp(EClient, EWrapper):
    """
    TestApp class combining EClient (for making API requests) and EWrapper (for handling callbacks).
    
    This sample specifically demonstrates how to request the market scanner parameters.
    """
    def __init__(self):
        # Initialize EClient with a reference to this instance as its wrapper.
        EClient.__init__(self, self)
        
    def nextValidId(self, orderId: int):
        """
        Callback method called by TWS when a valid order ID is received.
        
        This is our cue to begin sending our request—in this case, for scanner parameters.
        
        Args:
            orderId (int): The next valid order ID (not used here, but required for the callback).
        """
        # Request market scanner parameters. The result will be returned asynchronously
        # in the scannerParameters callback method.
        self.reqScannerParameters()

    def scannerParameters(self, xml: str):
        """
        Callback method that receives the scanner parameters as an XML string.
        
        Args:
            xml (str): The XML string containing available scanner parameters and their values.
            
        The XML is saved to a specified directory since it is too large for normal terminal output.
        """
        # Define the file path where the scanner parameters XML will be saved.
        file_path = "scanner.xml"
        
        # Open the file in write mode and save the XML string.
        with open(file_path, 'w') as file:
            file.write(xml)
        
        # Notify the user that the scanner parameters have been successfully received and saved.
        print("Scanner parameters received and saved to:", file_path)


In summary, the XML file is a comprehensive reference document that lists and describes:

- The available instruments and their types,

- The markets/exchanges (via location codes) you can target,

- The predefined scan codes (or scanning strategies) TWS offers, and

- A full set of filter keys (with default values and ranges) that let you refine your market scanning criteria.

This file is especially useful for developers as a “lookup table” when constructing scanner subscriptions so that you can experiment with different filters and ensure your API requests follow the allowed conventions defined by TWS.

In [3]:
"""
Main function for setting up the TestApp, connecting to TWS, and processing API data.

This function:
  - Instantiates the TestApp.
  - Connects to TWS/IB Gateway.
  - Starts the message loop (which processes callbacks, including our scanner parameter data).
"""
port = 7496  # Typical port for connecting to TWS (7496 for IB Gateway live trading)
clientId = 10

# Create an instance of the TestApp and connect to TWS.
app = TestApp()
app.connect("127.0.0.1", port, clientId)

# Begin processing messages from TWS (this call is blocking until disconnect).
app.run()

ERROR -1 1746631147343 2104 Market data farm connection is OK:cashfarm
ERROR -1 1746631147343 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 1746631147343 2104 Market data farm connection is OK:eufarm
ERROR -1 1746631147345 2104 Market data farm connection is OK:usopt
ERROR -1 1746631147345 2104 Market data farm connection is OK:usfarm
ERROR -1 1746631147345 2106 HMDS data farm connection is OK:euhmds
ERROR -1 1746631147345 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 1746631147345 2106 HMDS data farm connection is OK:ushmds
ERROR -1 1746631147345 2158 Sec-def data farm connection is OK:secdefeu


Scanner parameters received and saved to: scanner.xml


ERROR -1 1746650525013 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 1746650525013 2108 Market data farm connection is inactive but should be available upon demand.usopt
ERROR -1 1746651965164 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746651965646 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746651968638 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746651976135 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746651999872 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746652033105 2103 Market data farm connection is broken:cashfarm
ERROR -1 1746652092324 2104 Market data farm connection is OK:cashfarm
ERROR -1 1746687293601 2105 HMDS data farm connection is broken:euhmds
ERROR -1 1746687293644 2105 HMDS data farm connection is broken:ushmds
ERROR -1 1746687295895 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 1746687301422 21

The XML file includes the following:

Instrument Definitions

- Security Types: Definitions for all available instrument types (e.g., STK for stocks, OPT for options, FUT for futures, FX for forex, etc.).

- Descriptions: Details describing what each instrument type represents, which can help you determine which one to use in your subscriptions.

- Location Codes

- Market/Exchange Identifiers: A list of location codes that correspond to various markets or exchanges. Examples include “STK.US.MAJOR” (for major US exchanges like NYSE, NASDAQ, AMEX), “STK.EU” (for European exchanges), and codes for regions such as Asia or specific countries (like Hong Kong or Korea).

- Regional Information: Information that tells you which location codes correspond to major markets and any accompanying details on regional coverage.

- Scan Codes

Predefined Scan Types: A list of scan codes that define the type of market scanner you can use. Examples include “TOP_OPEN_PERC_GAIN,” “TOP_VOLUME,” “TOP_PCT_GAIN,” “TOP_PCT_LOSS,” etc. These codes determine the scan logic used by TWS to filter instruments based on market conditions.

- Filter Options (Tag Values)

Filter Keys: A comprehensive list of keys that you can use as filters when making a market scanner request, such as:

volumeAbove – to filter for securities with a trading volume above a specified threshold.

marketCapBelow or marketCapAbove – to filter based on market capitalization.

priceAbove or priceBelow – to filter out securities outside your desired price range.

Additional keys for metrics like average volume, option volume, 52-week high/low, earnings per share, etc.

- Metadata for Each Filter: For each filter key, the XML typically provides:

Default Values: A suggested value or the typical default used by TWS.

Allowable Ranges or Limits: Indicating what ranges or numeric limits you can apply.

Descriptions: Brief comments or documentation on what each filter does or what it signifies.

- Additional Configuration Parameters

Miscellaneous Details: There may be additional tags or configuration settings that inform you how TWS uses these parameters internally. This might include technical details or version-specific settings that are useful for advanced users.

Guidelines/Comments: The XML might include embedded commentary or guidelines that explain how to mix and match parameters for a valid scanner subscription.